# OsmRx example

## Imports and prepare input parameters

In [ ]:
from IPython.display import display

from bokeh.plotting import output_notebook

output_notebook()



In [ ]:
# Set a location name
location = "Roanne"


## Get POIs

In [ ]:
%%time
from osmrx import Pois

pois_object = Pois()
pois_object.from_location(location)

# Get the roads data: a list of dict containing the geometry and the attributes
pois_data = pois_object.data

## Get Roads

In [ ]:
%%time
from osmrx import Roads

# Let's to get the roads network and connect POIs found on the same location
vehicle_object = Roads("vehicle",
                        pois_object.data)
vehicle_object.from_location(location)

# Get the roads data: a list of dict containing the geometry and the attributes
roads_data = vehicle_object.data

## Display roads and nodes

In [ ]:
%%time

from bokeh.plotting import show
import geopandas as gpd
from gdf2bokeh import Gdf2Bokeh

map_session = Gdf2Bokeh(
    "My network map",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_dict_list(
    "Roads",
    roads_data,
    from_epsg=4326,
    color="black")

map_session.add_layer_from_dict_list(
    "POIs",
    pois_data,
    from_epsg=4326,
    color="blue", size=9)

map_session.add_layers_on_maps()

show(map_session.figure)


## Check topology details

In [ ]:
%%time

roads_topo_data = vehicle_object.topology_checker()

map_session = Gdf2Bokeh(
    "My topology network map",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_dict_list(
    "Roads unchanged",
    roads_topo_data.lines_unchanged,
    from_epsg=4326,
    color="black")

map_session.add_layer_from_dict_list(
    "Roads added",
    roads_topo_data.lines_added,
    from_epsg=4326,
    color="green")

map_session.add_layer_from_dict_list(
    "Roads split",
    roads_topo_data.lines_split,
    from_epsg=4326,
    color="orange")

map_session.add_layer_from_dict_list(
    "Intersections added",
    roads_topo_data.intersections_added,
    from_epsg=4326,
    color="red")

map_session.add_layer_from_dict_list(
    "POIs added",
    roads_topo_data.nodes_added,
    from_epsg=4326,
    color="blue",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)


## Compute a shortest path

In [ ]:
from osmrx.main.roads import GraphAnalysis


from_poi = pois_data[50]["geometry"]
to_poi = pois_data[60]["geometry"]

analysis_object = GraphAnalysis("pedestrian",
                                [from_poi, to_poi])
paths_found = analysis_object.get_shortest_path()
paths = [feature.path for feature in paths_found]

map_session = Gdf2Bokeh(
    "My computed path",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_geom_list(
    "Path found",
    paths,
    from_epsg=4326,
    color="blue")

map_session.add_layer_from_geom_list(
    "From",
    [from_poi],
    from_epsg=4326,
    color="red",
    size=9)

map_session.add_layer_from_geom_list(
    "To",
    [to_poi],
    from_epsg=4326,
    color="green",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)

## Compute a shortest path

In [ ]:
from bokeh.palettes import all_palettes

from_poi = pois_data[50]["geometry"]
 
analysis_object = GraphAnalysis("vehicle", [from_poi])
isochrones_built = analysis_object.isochrones_from_distance([0, 250, 500, 1000, 1500])

isochrones = isochrones_built.data
for pos, feature in enumerate(isochrones):
    feature["color"] = all_palettes['RdBu'][len(isochrones)][::-1][pos]

map_session = Gdf2Bokeh(
    "My isochrone",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_dict_list(
    "Isochrones",
    isochrones_built.data,
    from_epsg=4326,
    color="color",
    line_color="white",
    fill_alpha=0.6)

map_session.add_layer_from_geom_list(
    "From",
    [from_poi],
    from_epsg=4326,
    color="red",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)